## Preprocessing

In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# I've been running this on my laptop, which turns out to be faster than (free) Google Colab,
# and this shows whether I have it set up right to use the GPU:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [25]:
# Constants to toggle adjustments I've made to the cleanup
DROP_STATUS = False        # Maybe slightly harmful
DROP_INCOME_AMT = False    # Maybe slightly harmful
DROP_ASK_AMT = False       # Unclear
DROP_APP_TYPE = False      # Very harmful
DROP_CLASS = False         # Harmful
DROP_AFFILIATION = False   # Harmful
DROP_USE_CASE = False      # Harmful
DROP_ORGANIZATION = False  # Harmful
DROP_SPECIAL = True        # Maybe helpful

ADD_INCOME_MID = False           # Not helpful
ADD_INCOME_ASK_RATIO = False     # Not helpful

APPLICATION_CUTOFF = 100     # Originally 300
CLASSIFICATION_CUTOFF = 50   # Originally 1000

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
BUILD_NETWORK = not TUNE_NETWORK

# Random seed for train_test_split()
SPLIT_SEED = 8675309       # Originally 8675309
SPLIT_TEST_SIZE = 0.25     # Originally 0.25 (default)

# Since Accuracy is the metric referred to in the Challenge, it's the main metric used:
MAIN_ACCURACY_METRIC = "Accuracy"
LOSS_METRIC = "MeanSquaredError"

In [26]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
del application_df["EIN"]
del application_df["NAME"]

# Also drop STATUS since there only 5 non-active rows
if DROP_STATUS:
    del application_df["STATUS"]

if DROP_APP_TYPE:
    del application_df["APPLICATION_TYPE"]
if DROP_CLASS:
    del application_df["CLASSIFICATION"]
if DROP_AFFILIATION:
    del application_df["AFFILIATION"]
if DROP_USE_CASE:
    del application_df["USE_CASE"]
if DROP_ORGANIZATION:
    del application_df["ORGANIZATION"]
if DROP_SPECIAL:
    del application_df["SPECIAL_CONSIDERATIONS"]


In [27]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [28]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
if not DROP_APP_TYPE:
    v_counts = application_df["APPLICATION_TYPE"].value_counts()    
    print(v_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
if not DROP_APP_TYPE:
    application_types_to_replace = v_counts[v_counts < APPLICATION_CUTOFF].index.values
    
    # Replace in dataframe
    for app in application_types_to_replace:
        application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
    
    # Check to make sure replacement was successful
    print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64


In [30]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
if not DROP_CLASS:    
    v_counts = application_df["CLASSIFICATION"].value_counts()
    # Disabled because I'm tired of scrolling past it:
    print(v_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [31]:
# You may find it helpful to look at CLASSIFICATION value counts >1
if not DROP_CLASS:
    print(v_counts[v_counts > 1])


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [32]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
if not DROP_CLASS:
    classifications_to_replace = v_counts[v_counts < CLASSIFICATION_CUTOFF].index.values
    
    # Replace in dataframe
    for cls in classifications_to_replace:
        application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
    # Check to make sure replacement was successful
    print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: count, dtype: int64


In [33]:
# The INCOME_AMT column is categorical but could be numerical, which might be helpful.
if ADD_INCOME_MID:
    application_df["INCOME_MID"] = application_df["INCOME_AMT"].map({
        "0": 0,
        "25000-99999": 62499.5,
        "100000-499999": 299999.5,
        "1M-5M": 3000000,
        "1-9999": 5000,
        "10000-24999": 17499.5,
        "10M-50M": 30000000,
        "5M-10M": 7500000,
        "50M+": 50000000})
    print(application_df["INCOME_MID"].value_counts())

if DROP_INCOME_AMT:
    del application_df["INCOME_AMT"]


In [34]:
# Supplement the ask and income columns with a ratio
if ADD_INCOME_ASK_RATIO:
    application_df["INCOME_ASK_RATIO"] = application_df["INCOME_MID"] / application_df["ASK_AMT"]

# Surprisingly, the overwhelming about of loans are asking for the same amount of money, $5000,
# which could badly skew the model. So drop that column.
if DROP_ASK_AMT:
    del application_df["ASK_AMT"]

In [35]:
# Change the SPECIAL_CONSIDERATIONS column into something more easily usable
if not DROP_SPECIAL:
    application_df["SPECIAL_CONSIDERATIONS"] = application_df["SPECIAL_CONSIDERATIONS"].map({"Y":1, "N":0})

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_columns = []

if not DROP_INCOME_AMT:
    cat_columns.append("INCOME_AMT")
if not DROP_APP_TYPE:
    cat_columns.append("APPLICATION_TYPE")
if not DROP_CLASS:
    cat_columns.append("CLASSIFICATION")
if not DROP_AFFILIATION:
    cat_columns.append("AFFILIATION")
if not DROP_USE_CASE:
    cat_columns.append("USE_CASE")
if not DROP_ORGANIZATION:
    cat_columns.append("ORGANIZATION")

dummied_df = pd.get_dummies(application_df, columns=cat_columns)
dummied_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,...,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
1,1,108590,1,False,True,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
2,1,5000,0,True,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
3,1,6692,1,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [37]:
# Split our preprocessed data into our features and target arrays
X = dummied_df.loc[:, dummied_df.columns != "IS_SUCCESSFUL"]
y = dummied_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=SPLIT_TEST_SIZE, random_state=SPLIT_SEED)
print(X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Extra info to get a sense for how well it split:
for column in X.columns:
    print(f"X_train: {X_train[column].value_counts()}")
    print(f"X_test : {X_test[column].value_counts()}")
    print()

(34299, 52) (34299,) (25724, 52) (8575, 52) (25724,) (8575,)
X_train: STATUS
1    25721
0        3
Name: count, dtype: int64
X_test : STATUS
1    8573
0       2
Name: count, dtype: int64

X_train: ASK_AMT
5000       19080
6725           3
10478          3
6010           2
43814          2
           ...  
428551         1
12269          1
117502         1
17445          1
1296086        1
Name: count, Length: 6561, dtype: int64
X_test : ASK_AMT
5000      6318
23202        2
5706         2
24105        2
86039        2
          ... 
48866        1
298758       1
396368       1
58517        1
31927        1
Name: count, Length: 2249, dtype: int64

X_train: INCOME_AMT_0
True     18329
False     7395
Name: count, dtype: int64
X_test : INCOME_AMT_0
True     6059
False    2516
Name: count, dtype: int64

X_train: INCOME_AMT_1-9999
False    25169
True       555
Name: count, dtype: int64
X_test : INCOME_AMT_1-9999
False    8402
True      173
Name: count, dtype: int64

X_train: INCOME_AMT_10000

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((25724, 52), (8575, 52))

## Hyperband Search for Model Structure

In [39]:
%%time

MAX_EPOCHS_PER_MODEL = 15 # Meant to get a decent idea of parameter, not create a final model
HYPERBAND_ITERATIONS = 1  # "Number of times to iterate over the full Hyperband algorithm"
EXECUTIONS_PER_TRIAL = 2  # Training from scratch
SEARCH_FIT_EPOCHS = 15    # Epochs for each attempt to do a fit, I think. Not sure how this relates to MAX_EPOCHS_PER_MODEL.
OVERWRITE = True          # I'm hoping to be able to interrupt a run and resume it later

#ACTIVATIONS = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"]
#LAYER_COUNTS = [6, 12]
#UNITS_PER_LAYER_COUNTS = [30, 60]

#ACTIVATIONS = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"] -> Favorite: gelu
#ACTIVATIONS = ["tanh", "relu", "elu", "selu"]                                                 -> Favorite: tanh (surprise!)
#ACTIVATIONS = ["exponential", "mish", "relu6", "silu"]                                        -> Favorites: relu6 or mish, silu

ACTIVATIONS = ["tanh", "gelu", "mish", "relu6", "silu"]
LAYER_COUNTS = [1, 3, 9]
UNITS_PER_LAYER_COUNTS = [X.shape[1] * 2, X.shape[1], X.shape[1] // 2, X.shape[1] // 3]

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):

    nn = tf.keras.models.Sequential()

    # Input layer
    in_shape = (X.shape[1],)
    nn.add(tf.keras.Input(shape=in_shape))    

    # Hidden layers
    layer_count = hp.Choice(f'layer_count', LAYER_COUNTS)
    for l in range(layer_count):
        # Select activation and number of units for this layer
        activation = hp.Choice(f'activation_{l:04}', ACTIVATIONS)
        units =      hp.Choice(f'units______{l:04}', UNITS_PER_LAYER_COUNTS)

        # Add this layer
        nn.add(tf.keras.layers.Dense(units=units, activation=activation))
        


    # Output layer    
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss=LOSS_METRIC, optimizer='adam', metrics=[MAIN_ACCURACY_METRIC])
    
    return nn


# Run the kerastuner search for best hyperparameters
if TUNE_NETWORK:
    tuner = kt.Hyperband(
        create_model,
        objective=f"val_{MAIN_ACCURACY_METRIC}",
        max_epochs=MAX_EPOCHS_PER_MODEL,
        hyperband_iterations=HYPERBAND_ITERATIONS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        overwrite=OVERWRITE)
    tuner.search(X_train_scaled,y_train,epochs=SEARCH_FIT_EPOCHS,validation_data=(X_test_scaled,y_test))
    
    best_hyper = tuner.get_best_hyperparameters(1)[0]
    print(f"Best Hyper Values: {best_hyper.values}")
    
    best_model = tuner.get_best_models(1)[0]
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Best Model: Loss: {model_loss}, Accuracy: {model_accuracy}")


CPU times: user 15 μs, sys: 4 μs, total: 19 μs
Wall time: 20.7 μs


## Compile, Train and Evaluate the Model

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
if BUILD_NETWORK:
    in_shape = (X.shape[1],)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))

    # Hidden layers -- added iteratively
    # These activations showed up a lot in tuned model,
    # but really most of the available activations seem to get the job done:
    activations = ["gelu", "mish", "relu6", "tanh"]
    unit_counts = [X.shape[1]]    
    for u in unit_counts:
        for a in activations:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Check the structure of the model
    nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 52)             │         2,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 52)             │         2,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 52)             │         2,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 52)             │         2,756 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            53 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,077 (43.27 KB)

 Trainable params: 11,077 (43.27 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# Set up training checkpoint to save after each epoch, if it is a new best model:
BEST_PATH = './best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=BEST_PATH,
    monitor=MAIN_ACCURACY_METRIC,
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1)


In [47]:
%%time

#
# Train the network
# Run this block repeatedly to keep going, but be aware that if you change other code you may need to
# re-create the network before you get the combination of data and logic you want.
#

EPOCHS = 15             # Epochs per fit() call
START_WITH_BEST = True  # Whether to load the previously saved "best" model before fitting here
REVERT_TO_BEST = True   # Whether to load the "best" model after fitting, discarding more recent but less accurate models

if START_WITH_BEST:
    nn = tf.keras.models.load_model(BEST_PATH)

if BUILD_NETWORK:    
    # Iterate through learning rates to allow larger and smaller increments adjusting weights
    for LR in [0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.005, 0.05, 0.1, 0.5]:
        # Compile the model
        nn.compile(loss=LOSS_METRIC, optimizer=tf.keras.optimizers.SGD(learning_rate=LR), metrics=[MAIN_ACCURACY_METRIC, "F1Score", "AUC"])
        
        # Train the model
        print(F"LR: {LR}")
        nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=[model_checkpoint_callback])
    
        # Change the accuracy metric back to simply "accuracy"
        nn.compile(loss=LOSS_METRIC, metrics=[MAIN_ACCURACY_METRIC])
        model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
        print(f"Final Model, Test Set results: Loss: {model_loss}, Accuracy: {model_accuracy}")

        # Show the metrics from the best known model
        best_model = tf.keras.models.load_model(BEST_PATH)
        best_model.compile(loss=LOSS_METRIC, metrics=[MAIN_ACCURACY_METRIC])    
        model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
        print(f"Best  Model, Test Set results: Loss: {model_loss}, Accuracy: {model_accuracy}")
    
        if REVERT_TO_BEST:
            nn = best_model


LR: 0.5
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7873 - Accuracy: 0.7311 - F1Score: 0.6974 - loss: 0.1850
Epoch 1: Accuracy did not improve from 0.73974
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - AUC: 0.7873 - Accuracy: 0.7311 - F1Score: 0.6974 - loss: 0.1850
268/268 - 1s - 2ms/step - Accuracy: 0.7257 - loss: 0.1845
Final Model, Test Set results: Loss: 0.18450184166431427, Accuracy: 0.7257142663002014
268/268 - 1s - 2ms/step - Accuracy: 0.7354 - loss: 0.1821
Best  Model, Test Set results: Loss: 0.18213044106960297, Accuracy: 0.7353935837745667
LR: 0.1
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7967 - Accuracy: 0.7340 - F1Score: 0.6983 - loss: 0.1806
Epoch 1: Accuracy did not improve from 0.73974
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - AUC: 0.7967 - Accuracy: 0.7340 - F1Score: 0.6983 - loss: 0.1806
268/268 - 1s - 2ms/step - Accuracy: 0.7381 - loss: 0.1820
Final Model, Test Set results: Loss: 0.1819944530725479, Accuracy: 0.7380757927894592
268/268 - 1s - 2ms/step - Ac

In [49]:
# Export our model to HDF5 file (which is obsolete to save as .keras too!)
# Uncomment these to update the stored model:
#nn.save("AlphabetSoupCharity_Optimization.h5")
#nn.save("AlphabetSoupCharity_Optimization.keras")